In [1]:
import pandas as pd

def get_training_data(smol=0.01, seed=1337):
    h1b_data = pd.read_csv('data.csv')
    desired_cols = ['JOB_TITLE',
        'EMPLOYER_NAME',
        'WORKSITE_STATE',
        'WORKSITE_CITY',
        'WAGE_RATE_OF_PAY_FROM']
    a = h1b_data[desired_cols]
    a['WAGE_RATE_OF_PAY_FROM'] = a['WAGE_RATE_OF_PAY_FROM'].apply(lambda x: int(x.replace(',','')[:-3]))
    b = a[~(a['WAGE_RATE_OF_PAY_FROM'] < 10000)]
    b = b.dropna()
    c = b.copy()
    TECHIES = set(['SOFTWARE', 'PROGRAMMER', 'DEVELOPER', 'ENGINEER'])
    from numpy.random import rand
    is_a_techie = lambda job_title: any([word in TECHIES for word in job_title.upper().strip().split()]) and rand() <= 0.95
    c['HENRY'] = c['JOB_TITLE'].map(is_a_techie)
    return c.sample(frac=smol, replace=False, random_state=1337)

train = get_training_data(0.001)
print(len(train))
train.head()

/Users/rainierababao/anaconda2/envs/nn/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/rainierababao/anaconda2/envs/nn/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


495


,JOB_TITLE,EMPLOYER_NAME,WORKSITE_STATE,WORKSITE_CITY,WAGE_RATE_OF_PAY_FROM,HENRY
79289,COMPUTER PROGRAMMER/CONFIGURER 2,ACCENTURE LLP,IN,INDIANAPOLIS,61100,False
152540,EMBEDDED SOFTWARE ENGINEER,"PURIGEN BIOSYSTEMS, INC.",CA,PLEASANTON,110000,True
187354,ACCOUNT MANAGER,"GRAPESEED MEDIA, LLC",NY,NEW YORK,80000,False
386514,ASSOCIATE CONSULTANT: PROJECT,"CSC CONSULTING, INC.",TX,AUSTIN,53206,False
452448,ASSISTANT PROFESSOR,TEXAS A&M UNIVERSITY - COMMERCE,TX,COMMERCE,51095,False


In [2]:
train['JOB_TITLE'].describe()

count                    495
unique                   320
top       PROGRAMMER ANALYST
freq                      49
Name: JOB_TITLE, dtype: object

In [3]:
from gensim.models import Word2Vec

In [48]:
h1b_data = pd.read_csv('data.csv')
desired_cols = ['JOB_TITLE',
    'EMPLOYER_NAME',
    'WORKSITE_STATE',
    'WORKSITE_CITY',
    'WAGE_RATE_OF_PAY_FROM']
a = h1b_data[desired_cols]

/Users/rainierababao/anaconda2/envs/nn/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [151]:
# a['JOB_TITLE']
# a['WORKSITE_CITY'].describe()

job_freqs = a[['JOB_TITLE', 'WORKSITE_CITY']].groupby('WORKSITE_CITY').count()
# print(job_freqs)

                                JOB_TITLE
WORKSITE_CITY                            
# 47644 MI METRO DIVISION               1
#11460 ANN ARBOR                        2
#19100 COLLIN MSA                       1
#19100 DALLAS MSA                       1
#19124 DALLAS-PLANO-IRVING, TX          3
#19820 DETROIT MSA                     12
#2400 1ST FLOOR                         1
#29G                                    1
#41740 SAN DIEGO CA MSA                 1
#47644 MI METRO DIVISION               11
#47644 MICH METRO DIVISION              2
&NBSP;IRVING                            1
, JERSEY CITY                           1
, RICHARDSON                            1
, SAN JOSE,                             1
, SUNNYVALE                             1
,HELENA                                 1
,MAUMEE                                 1
1 BAXTER PKWY                           1
1 RIVER ROAD, BLDG-43                   1
100 EAST 77TH STREET                    1
100 PROGRESS PARKWAY              

In [161]:
top_96 = job_freqs[job_freqs['JOB_TITLE']>1000].sort_values('JOB_TITLE', ascending=False)

In [174]:
pa_jobs = a[a['JOB_TITLE']=='BUSINESS ANALYST'][['JOB_TITLE', 'WORKSITE_CITY']].groupby('WORKSITE_CITY').count()
pa_jobs['WORKSITE_CITY'] = pa_jobs.index
top_96['WORKSITE_CITY'] = top_96.index

print(len(a[a['JOB_TITLE']=='BUSINESS ANALYST']))

pa_jobs.head()

6466


,JOB_TITLE,WORKSITE_CITY
WORKSITE_CITY,,
"1000 SKOKIE BLVD., SUITE 225",1,"1000 SKOKIE BLVD., SUITE 225"
15TH FLOOR,1,15TH FLOOR
2200 MASTERCARD BLVD,1,2200 MASTERCARD BLVD
4600 SILICON DRIVE,1,4600 SILICON DRIVE
800 N GLEBE RD #500,1,800 N GLEBE RD #500


In [173]:
pd.merge(top_96, pa_jobs, how='left', on='WORKSITE_CITY')

,JOB_TITLE_x,WORKSITE_CITY,JOB_TITLE_y
0,31956,NEW YORK,301
1,13494,SAN FRANCISCO,92
2,11639,HOUSTON,145
3,9959,CHICAGO,92
4,9356,ATLANTA,107
5,9273,SAN JOSE,65
6,7674,IRVING,124
7,6952,SUNNYVALE,31
8,6935,DALLAS,139
9,6792,SEATTLE,23


In [77]:
sentences=map(str, a['JOB_TITLE'].tolist())
print(list(sentences)[:10])
# model = Word2Vec(sentences=sentences)
model = Word2Vec(zip(sentences, ["sentence"]*len(list(sentences))), size=2, min_count=0)
# model.build_vocab(sentences=sentences, keep_raw_vocab=True)

model.wv.vocab

['ASSOCIATE DATA INTEGRATION', 'SENIOR ASSOCIATE', '.NET SOFTWARE PROGRAMMER', 'PROJECT MANAGER', 'ASSOCIATE - ESOTERIC ASSET BACKED SECURITIES', 'CREDIT RISK METRICS SPECIALIST', 'BUSINESS SYSTEMS ANALYST', 'PROGRAMMER ANALYST', 'PROGRAMMER ANALYST', 'PROGRAMMER ANALYST']


RuntimeError: you must first build vocabulary before training the model

In [137]:
import gensim
# sentences = ['ASSOCIATE DATA INTEGRATION', 'SENIOR ASSOCIATE', '.NET SOFTWARE PROGRAMMER', 'PROJECT MANAGER', 'ASSOCIATE - ESOTERIC ASSET BACKED SECURITIES', 'CREDIT RISK METRICS SPECIALIST', 'BUSINESS SYSTEMS ANALYST', 'PROGRAMMER ANALYST', 'PROGRAMMER ANALYST', 'PROGRAMMER ANALYST']
sentences = list(map(lambda x: [str(x)], a['JOB_TITLE'].tolist()))
sentence_tokens = list(map(lambda x: str(x).replace(',','').split(), a['JOB_TITLE'].tolist()))
print(sentences[:5])
print(sentence_tokens[:5])
# train word2vec on the two sentences
# model = gensim.models.Word2Vec(sentences, min_count=1)

# model = gensim.models.Word2Vec(zip(sentences, ["sentence"]*len(list(sentences))), size=2, min_count=0, window=1, sg=0, negative=4)
# model = gensim.models.Word2Vec(sentences, size=2, min_count=0, window=1, sg=0, negative=4)
model = gensim.models.Word2Vec(min_count=0, window=1, sg=0)
model.build_vocab(sentence_tokens)
# model.build_vocab(sentences)
model.train(sentences, total_examples=len(sentences), epochs=5)

# model = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)



[['ASSOCIATE DATA INTEGRATION'], ['SENIOR ASSOCIATE'], ['.NET SOFTWARE PROGRAMMER'], ['PROJECT MANAGER'], ['ASSOCIATE - ESOTERIC ASSET BACKED SECURITIES']]
[['ASSOCIATE', 'DATA', 'INTEGRATION'], ['SENIOR', 'ASSOCIATE'], ['.NET', 'SOFTWARE', 'PROGRAMMER'], ['PROJECT', 'MANAGER'], ['ASSOCIATE', '-', 'ESOTERIC', 'ASSET', 'BACKED', 'SECURITIES']]


87942

In [138]:
print(model.wv['ENGINEER'])
# print(dir(model))
model.predict_output_word(['VP'])
# print(model.wv['VP,'])
# print(model.wv['VP, SENIOR PREPARER OF FINANCIAL STATEMENTS'])
# model.most_similar('SOFTWARE DEVELOPER')

[ -1.67621917e-03  -4.96128527e-03  -3.73596256e-03  -7.65823759e-04
  -2.45247525e-03   2.75240256e-03   3.58636910e-03   3.37061239e-03
   3.94580886e-03   3.64934839e-03   8.88323702e-04  -3.01647611e-04
  -3.65966279e-03  -1.76443020e-03   3.26544465e-03  -4.11040848e-03
   4.59641888e-04   1.23630755e-03  -8.51580000e-04   4.03637160e-03
   1.26522989e-03  -2.32812902e-03   1.64826075e-03  -1.31431222e-03
  -1.09429564e-03  -1.56679458e-03   2.96209753e-03   2.24677427e-03
  -3.88907269e-03   9.00854531e-04  -4.51103318e-03   4.71678982e-03
   1.54093327e-03   8.52491183e-04  -2.09980179e-03   4.74854745e-03
   1.37284966e-04   1.28949934e-03  -1.76038267e-03  -2.41631758e-03
  -3.82784847e-03  -6.77138916e-04  -3.84863419e-03  -4.54647141e-03
   2.45157653e-03  -1.46395888e-03   2.80779414e-03  -7.37971161e-04
  -3.67942872e-03  -4.96031623e-03  -4.70154965e-03  -3.41084716e-03
  -6.54278090e-04  -3.39915114e-03  -4.98287193e-03  -2.16506631e-03
   4.25567821e-04  -2.04488332e-03

[('EXOTICS', 4.8407397e-05),
 ('(VETERINARIAN)', 4.8407397e-05),
 ('1615.25985', 4.8407397e-05),
 ('CENTRIFY', 4.8407397e-05),
 ('1615.22565', 4.8407397e-05),
 ('1615.20597', 4.8407397e-05),
 ('LL)', 4.8407397e-05),
 ('1615.5961', 4.8407397e-05),
 ('RESEARCHER-VEHICLE', 4.8407397e-05),
 ('DIRECTOR--AESTHETIC', 4.8407397e-05)]

In [18]:
model.save('job2vec.model')

In [19]:
model = Word2Vec.load('job2vec.model')

In [35]:
print(model)
print(dir(model))
# print(model.vocab.keys())
# model.scan_vocab(['engineer'])
print(model.raw_vocab)
# model.wv['ENGINEER']


Word2Vec(vocab=52, size=10, alpha=0.025)
['__class__', '__contains__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', '_adapt_by_suffix', '_do_train_job', '_load_specials', '_minimize_model', '_raw_word_count', '_save_specials', '_smart_save', 'accuracy', 'alpha', 'batch_words', 'build_vocab', 'cbow_mean', 'clear_sims', 'compute_loss', 'corpus_count', 'create_binary_tree', 'cum_table', 'delete_temporary_training_data', 'doesnt_match', 'estimate_memory', 'evaluate_word_pairs', 'finalize_vocab', 'get_latest_training_loss', 'hashfxn', 'hs', 'init_sims', 'initialize_word_vectors', 'intersect_word2vec_format', 'iter', 'layer1_size', 'load', 'load_word2vec_format', 'log_accuracy', 